<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
# Upgrading seaborn
! pip install --upgrade seaborn

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.linear_model import LinearRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor


from scipy.stats import boxcox
from scipy.stats import yeojohnson, uniform
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, truncnorm, randint
from pprint import pprint
import random
from catboost import CatBoostRegressor
from itertools import groupby

import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor

from sklearn.metrics import recall_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.base import clone
import datetime
from datetime import datetime, timedelta
import math

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from pprint import pprint

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
print(sns.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 16
DIR_TRAIN  = '../input/parsedcars/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'car_prices_new.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

## Предобработка данных 

In [ ]:
train.head(5)

In [ ]:
train.drop(columns = ['Unnamed: 0', 'car_url'], inplace= True)

In [ ]:
test.drop(columns = ['car_url', 'color', 'complectation_dict', 'complectation_dict', 'description', 'equipment_dict', 'complectation_dict', 'image', 'modelDate', 'model_info', 'vehicleConfiguration', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Владение' ], inplace= True)

#### Проведем EDA наших данных, которые содержатся в тренинговой и тестовой выборках

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
test['engineDisplacement'].value_counts()

In [ ]:
train['engineDisplacement'].value_counts()

In [ ]:
# Очистим признак engineDisplacement, удалив из него лишние символы и конвертировав в число
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: x.replace(' LTR', ''))

In [ ]:
test['engineDisplacement'].value_counts()

Как видно из анализа признака, в нем присутствует 55 пустых значений. Скорее всего, эти пропуски появились из-за присутствия в датасете электромобилей, у которых нет такого объема двигателя

In [ ]:
# Для начала заменим пустые значения признака engineDisplacement на 0
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: '0.0' if x == '' else x)

In [ ]:
# Поскольку в тренинговом датасете я сразу заменял пустые значения на ноль, повторим эту процедуру для тренингового датасета
train['engineDisplacement'] = train['engineDisplacement'].apply(lambda x: '0.0' if x == '' else x)

In [ ]:
# Конвертируем строковій значения призанака engineDisplacement в float
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: float(x))

Теперь нужно определиться с тем, что делать с пропущенными значениями engineDisplacement. Заменим их на среднее значение.

In [ ]:
# Вычислим среднее значение признака engineDisplacement
sum = 0
count = 0
for item in test['engineDisplacement']:
    if item != 0.0:
        sum = sum + item
        count +=1
print(sum/count)

In [ ]:
# Заменим пустые(нулевые) значения признака на средние значения в поочередно в тренинговом и тестовом датасетах
train['engineDisplacement'] = train['engineDisplacement'].apply(lambda x: 2.4 if x == 0.0 else x)

In [ ]:
# Replacing empty values for engine displacement with average engine displacement
test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: 2.4 if x == 0.0 else x)

In [ ]:
test['engineDisplacement'].value_counts()

In [ ]:
train['engineDisplacement'].value_counts()

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
# Выделим целевую переменную из тренингового датасета
y = train['price']
train.drop(columns= ['price'], inplace = True)

In [ ]:
# Введем дополнительный признак sample для различия тренингового и тестового датасета, после их обїединения в один датафрейм
train['sample'] = 1
test['sample'] = 0

In [ ]:
# Объеденим тренинговый и тестовый датасеты для последующей обработки данных
data = train.append(test, sort=False).reset_index(drop=True) 

In [ ]:
data.tail()

Продолжим обработку данных уже объединенного датасета

In [ ]:
data['bodyType'].value_counts()

In [ ]:
data['brand'].value_counts()

In [ ]:
# Преобразуем значения признака Brand таким образом, чтобы избежать повторов из-за разных вариантах написания 
data['brand'] = data['brand'].apply(lambda x: 'mercedes' if x == 'Mercedes-Benz' else x)

In [ ]:
data['brand'] = data['brand'].apply(lambda x: x.lower())

In [ ]:
data['brand'].value_counts()

У нас в датасете присутствуют автомобили лишь 12-ти разных брендов

In [ ]:
data['enginePower'].value_counts()

In [ ]:
# Очистим признак enginePower (Мощность двигателя), убрав лишние символы и конвертировав его в числовое значение
data['enginePower'] = data['enginePower'].apply(lambda x: x.replace(' N12', ''))

In [ ]:
data['enginePower'] = data['enginePower'].apply(lambda x: float(x))

In [ ]:
data['enginePower'].value_counts()

In [ ]:
data['fuelType'].value_counts()

In [ ]:
# Преобразуем значения признака fuelType(тип дивгателя) таким образом, чтобы избежать повторов из-за разных вариантах написания
data['fuelType'] = data['fuelType'].apply(lambda x: x.lower())

In [ ]:
data['fuelType'].value_counts()

In [ ]:
data['mileage'].value_counts()

In [ ]:
data['model_name'].value_counts()

In [ ]:
data['name'].value_counts()

In [ ]:
data['productionDate'].value_counts()

In [ ]:
data['vehicleTransmission'].value_counts()

In [ ]:
data.info()

In [ ]:
# Переименуем названия признаков на руском язіке в английский для консистентности
data.rename(columns={"Владельцы": "owners", "ПТС": "passport", "Привод": "driveType", "Руль": "wheel", "Состояние": "condition", "Таможня": "custom"}, inplace = True)

In [ ]:
data.head()

In [ ]:
data['owners'].value_counts()

In [ ]:
# Избавимся от лишних символов и преобразуем в числа значения признака Owners (Количество владельцев)
data['owners'] = data['owners'].apply(lambda x: x[0:1])

In [ ]:
data['owners'].value_counts()

In [ ]:
data['passport'].value_counts()

In [ ]:
data[data['passport'].isnull()]

In [ ]:
# У признака Passport, есть одно пропущенное значение. Заменим его наиболее встречающимся значением
data.at[28965,'passport'] = 'Оригинал'

In [ ]:
data['driveType'].value_counts()

In [ ]:
data['wheel'].value_counts()

In [ ]:
data['condition'].value_counts()

In [ ]:
data['custom'].value_counts()

In [ ]:
# Поскольку признаки Condition (Состояние) и Custom (Таможня) принимают единственно значение, удалим их из датасета, т.к. они не окажут положительного влияние на обучение модели
data.drop(columns = ['condition'], inplace = True)
data.drop(columns = ['custom'], inplace = True)

In [ ]:
# Также предлагаю удалить признак Name, т.к. он принимает очень большое количество значений
data.drop(columns = ['name'], inplace = True)

In [ ]:
data.info()

#### Итого, на данный момент у нас в объединенном датасете осталось 13 признаков (плюс один признак индикатор трейна или теста)

#### Разобъем все признаки по типам хранящихся в них данных

In [ ]:
data.head()

In [ ]:
columns = data[['bodyType', 'brand', 'engineDisplacement', 'enginePower', 'fuelType', 'mileage', 'model_name', 'productionDate', 'vehicleTransmission', 'owners', 'passport', 'driveType', 'wheel']].columns.tolist()

In [ ]:
# Категориальные признаки
cat_cols = data[['bodyType', 'brand', 'fuelType', 'model_name']].columns.tolist()

In [ ]:
# Числовые признаки
num_cols = data[['engineDisplacement', 'enginePower', 'mileage', 'productionDate']].columns.tolist()

In [ ]:
# Бинарные признаки
bin_cols = data[['vehicleTransmission', 'owners', 'passport', 'driveType', 'wheel']].columns.tolist()
# Для удобства я включил признак Owners(Количетсво владельцев) в список бинарных переменных, несмотря на то, что это - ординальная переменная, которая принимает 3 значения

## Анализ Данных

#### Проанализируем для начала числовые признаки

In [ ]:
# Выделим числовые признаки в отдельный датасет
numdata = data[num_cols]

In [ ]:
# Построим графики распределния числовіх признаков
fig, axes = plt.subplots(2, 2, figsize=(30,20))
axes = axes.flatten() # переводим двумерный массив осей в одномерный для корректной индексации

i = 0
for col in num_cols:    
    ax = sns.histplot(x = col, 
                data=numdata.loc[numdata.loc[:, col].isin(numdata.loc[:, col].value_counts().index[:])],
               ax=axes[i], bins = 150)
    plt.xticks(rotation=45)        
    i+=1
    
fig.tight_layout()

#### Как видно из диаграмм, все числовые характеристики имеют распределение близкое к нормальному. Однако практически у всех характеристик имеется хвост справа или слева. Попробуем логарифмировать признаки и посмотрим, как изменится их распределение


In [ ]:
numdata_log = numdata.copy()

In [ ]:
for col in num_cols:
    numdata_log[col] = numdata[col].apply(lambda x: math.log(x) if x > 0 else x)

#### Построим графики повторно

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(30,20))
axes = axes.flatten() # переводим двумерный массив осей в одномерный для корректной индексации

i = 0
for col in num_cols:    
    ax = sns.histplot(x = col, 
                data=numdata_log.loc[numdata_log.loc[:, col].isin(numdata_log.loc[:, col].value_counts().index[:])],
               ax=axes[i], bins = 150)
    plt.xticks(rotation=45)        
    i+=1
    
fig.tight_layout()

В целом, распределение признаков engineDisplacement и enginePower немного улучшилось(стало ближе к нормальному) после логарифмирования. А распределение признаков mileage и productionDate не улучшилось. Пока обойдемся без логарифмирования.

#### Теперь посмотрим на корреляции между числовыми признаками

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1)

In [ ]:
data[num_cols].corr().abs()

#### Видим, что есть большая корреляция между признаками enginePower и engineDisplacement. Значит один из этих признаков стоит исключить из модели, чтобы избежать переобучения. Исключим engineDisplacement(объем двигателя), т.к. этой характеристики нет у электромобилей.

In [ ]:
num_cols.remove('engineDisplacement')

#### Тут стоит отметить, что для использования некоторых моделей нам нужно будет отдельно предобрабатывать призаки разных типов и затем объединять их в единый массив для передачи в модель

#### Поработаем с бинарными переменными. Переведем их в числа с помощью метода Label Encoding

In [ ]:
# Для начала создадим копию объединенного датасета и будем работать с ней
datacopy = data.copy()

In [ ]:
label_encoder = LabelEncoder()

for column in bin_cols:
    print(column)
    datacopy[column] = label_encoder.fit_transform(datacopy[column])
    
# убедимся в преобразовании    
datacopy.head()

In [ ]:
#new_cat_cols = cat_cols

In [ ]:
#new_cat_cols

#### Теперь поработаем с категориальными переменными. Переведем их в числа с помощью метода One Hot Encoding. 

In [ ]:
X_cat = OneHotEncoder(sparse = False).fit_transform(datacopy[cat_cols].values)

In [ ]:
X_cat.shape

In [ ]:
#Стандартизируем числовые переменные
X_num = StandardScaler().fit_transform(datacopy[num_cols].values)

In [ ]:
X_num.shape

In [ ]:
datacopy[bin_cols].values.shape

In [ ]:
datacopy['sample'].shape

In [ ]:
# Адаптируем столбец sample таким образом, чтобы его можно было присоединить к объединенному массиву
sample = np.reshape(datacopy['sample'].values,(datacopy['sample'].values.size, 1))

In [ ]:
sample.shape

In [ ]:
# Объединяем уже предобработанные данные в единый массив, пригодный для передачи в обучающие модели
X = np.hstack([X_num, datacopy[bin_cols].values, X_cat, sample])

In [ ]:
X

In [ ]:
# Теперь для удобства преобразуем получившийся массив обратно в датафрейм
df = pd.DataFrame(X)

In [ ]:
df.head()

In [ ]:
# К сожалению, у нас теперь нет наименований колонок, но мы помним, что признак sample, 
# который отвечал за отличие тренинговой и тестовой частей датасета должен был быть последней колонкой. Проверим это
df.iloc[:, 921].value_counts()

In [ ]:
# Выделяем тренинговую часть датасета
X_1 = df[df.iloc[:, 921] == 1.0]

In [ ]:
X_1

In [ ]:
# Проверяем, что размерность тренинговой части датасета и целевой переменной соответствуют друг другу
y.shape

In [ ]:
# Выделяем тестовую часть датасета
X_2 = df[df.iloc[:, 921] == 0.0]

In [ ]:
X_1

In [ ]:
# Разбиваем тренинговую часть датасета на собственно тренинговую и валидационную части для обучения некоторых типов моделей
X_train, X_valid, y_train, y_valid = train_test_split(X_1, y, test_size=VAL_SIZE, shuffle = True, random_state=RANDOM_SEED)

## Model1. Создадим "наивную" модель
Эта модель будет предсказывать среднюю цену по модели двигателя (enginePower). C ней будем сравнивать другие модели.

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам enginePower в трейне и размечаем тест
predict = X_valid.iloc[:, 0].map(tmp_train.groupby(0)['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_valid, predict.values))*100:0.2f}%")

## Model2. Создадим модель на основе линейной регрессии

In [ ]:
linreg = LinearRegression().fit(X_train, y_train)

In [ ]:
y_pred = linreg.predict(X_valid)
print(f"Точность модели регрессии по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

#### Как видим, точность модели линейной регресси по метрике MAPE крайне плохая

#### Проведем регуляризацию

In [ ]:
# Создадим гиперпараметры
hyperparameters = {"fit_intercept": [True, False],
              "normalize": [True, False]
             }

model = LinearRegression()

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=10, n_jobs=-1)

best_model = clf.fit(X_train, y_train)

# View best hyperparameters
print('Лучшее Fit_Intercept:', best_model.best_estimator_.get_params()['fit_intercept'])
print('Лучшее Normalize:', best_model.best_estimator_.get_params()['normalize'])

#### Попробуем линейную регрессию с гиперпараметрами

In [ ]:
linreg2 = LinearRegression(fit_intercept = True, normalize = False).fit(X_train, y_train)

In [ ]:
y_pred2 = linreg2.predict(X_valid)
print(f"Точность наивной модели по метрике MAPE: {(mape(y_valid, y_pred2))*100:0.2f}%")

#### К сожалению, у модели Линейной Регрессии не так много гиперпараметров, которые можно настраивать. Поэтому даже с их настройкой ожидаемо ничего кардинально не поменялось

## Model2_1. Создадим модель на основе линейной регрессии используя только числовые признаки

In [ ]:
data.head()

In [ ]:
# Создадим копию датасета для работы с ним
df_lin = data[num_cols + ['sample']]

In [ ]:
df_lin

In [ ]:
# Выделяем из предобработанного датасета тренинговую и тестовую части на основе признака sample
X1 = df_lin.query('sample == 1').drop(['sample'], axis=1)
X2 = df_lin.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
# Разбиваем тренинговый датасет на собственно тренинговую и валидационную части для алгоритма CatBoost
X_train_lin, X_valid_lin, y_train_lin, y_valid_lin = train_test_split(X1, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
linreg3 = LinearRegression(fit_intercept = True, normalize = False).fit(X_train_lin, y_train_lin)

In [ ]:
y_pred3 = linreg3.predict(X_valid_lin)
print(f"Точность наивной модели по метрике MAPE: {(mape(y_valid_lin, y_pred3))*100:0.2f}%")

#### Как видим, точность модели Линейной Регрессии все еще остается крайне плохой, но это уже не такие сумасшедшие показатели, как были при предыдущих итерациях. Но факт, остается фактом, нужно пробовать другие алгоритмы

## Model3. Создадим модель на основе CatBoost

#### Для модели CatBoost будем использовать оригинальную версию датасета, т.к. эта модель "заточена" под работу с данными разных типов, поэтому предварительная предобработка данных тут не требуется

In [ ]:
data.head()

In [ ]:
# Создадим копию датасета для работы с ним
df_cars = data[cat_cols + num_cols + bin_cols + ['sample']]

In [ ]:
df_cars

In [ ]:
nonnumcols = cat_cols + bin_cols

In [ ]:
for colum in nonnumcols :    
    df_cars[colum] = df_cars[colum].astype('category').cat.codes

In [ ]:
df_cars.info()

In [ ]:
df_cars.head()

In [ ]:
# Выделяем из предобработанного датасета тренинговую и тестовую части на основе признака sample
X1 = df_cars.query('sample == 1').drop(['sample'], axis=1)
X2 = df_cars.query('sample == 0').drop(['sample'], axis=1)

In [ ]:
X1

In [ ]:
# Разбиваем тренинговый датасет на собственно тренинговую и валидационную части для алгоритма CatBoost
X_train_c, X_valid_c, y_train_c, y_valid_c = train_test_split(X1, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Запускаем алдгоритм CatBoost с параметрами по умолчанию
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train_c, y_train_c,
         #cat_features=cat_features_ids,
         eval_set=(X_valid_c, y_valid_c),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.save_model('catboost_single_model_baseline.model')

In [ ]:
predict = model.predict(X_valid_c)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_valid_c, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 14,36%!

In [ ]:
# Попробуем поиграться с параметрами CatBoost
model_2 = CatBoostRegressor(iterations = 5000,
                          learning_rate = 0.05,
                          l2_leaf_reg = 10,
                          bagging_temperature = 1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          leaf_estimation_method = 'Newton',
                          leaf_estimation_iterations = 10                          
                         )

model_2.fit(X_train_c, y_train_c,
         #cat_features=cat_features_ids,
         eval_set=(X_valid_c, y_valid_c),
         verbose_eval=1000,
         use_best_model=True,
         plot=True
         )

model_2.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_2 = model_2.predict(X_valid_c)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_valid_c, predict_2))*100:0.2f}%")

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
np.log(y_train_c)

In [ ]:
model_3 = CatBoostRegressor(iterations = 5000,
                          learning_rate = 0.05,
                          l2_leaf_reg = 10,
                          bagging_temperature = 1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          leaf_estimation_method = 'Newton',
                          leaf_estimation_iterations = 10 
                         )

model_3.fit(X_train_c, np.log(y_train_c),
         #cat_features=cat_features_ids,
         eval_set=(X_valid_c, np.log(y_valid_c)),
         verbose_eval=1000,
         use_best_model=True,
         #plot=True
         )

model_3.save_model('catboost_single_model_3_baseline.model')

In [ ]:
predict_test = np.exp(model_3.predict(X_valid_c))
#predict_submission = np.exp(model.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_valid_c, predict_test))*100:0.2f}%")

Как видим точность возросла до 12.46%, а что будет на ЛБ?

In [ ]:
predict_submission = np.exp(model_3.predict(X2))

# Submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 19.9%** на LeaderBoard соревнования в Kaggle!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

## Теперь попробуем некоторые другие алгоритмы

In [ ]:
# Градиентный бустинг
model_gbr = GradientBoostingRegressor(n_estimators=300)
model_gbr.fit(X_train, y_train)
y_pred_gbr = model_gbr.predict(X_valid)
print(f"Точность модели GradientBoostingRegressor по метрике MAPE: {(mape(y_valid, y_pred_gbr))*100:0.2f}%")

In [ ]:
predict_submission_gbr = model_gbr.predict(X_2)

In [ ]:
sample_submission['price'] = predict_submission_gbr
sample_submission.to_csv(f'submission_gbr.csv', index=False)
sample_submission.head(10)

На LeaderBoard получили **MAPE 27%**, что намного хуже, чем результат для CatBoosting

In [ ]:
# Decision Tree Regressor
model_dectree = DecisionTreeRegressor(random_state = RANDOM_SEED)
model_dectree.fit(X_train, y_train)
y_pred_dectree = model_dectree.predict(X_valid)
print(f"Точность модели DecisionTreeRegressor по метрике MAPE: {(mape(y_valid, y_pred_dectree))*100:0.2f}%")

In [ ]:
# Попробуем поиграться с гиперпараметрами для модели Decision Tree

parameters={"splitter":["best","random"],
            "max_depth" : [7,9,11,12],
           "min_samples_leaf":[1,2,3,6,7,8],
           "min_weight_fraction_leaf":[0.1,0.3, 0.5],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20,30,40,50] }

In [ ]:
grid_search_dectree=GridSearchCV(model_dectree,param_grid=parameters,cv=3, n_jobs=-1)
grid_search_dectree.fit(X_train, y_train)

In [ ]:
# лучшие гиперпараметры
grid_search_dectree.best_params_

In [ ]:
# Обучим модель Decision Tree Model с гиперпараметрами
model_dectree2 = DecisionTreeRegressor(max_depth = 7, min_weight_fraction_leaf = 0.1, splitter ='best', random_state = RANDOM_SEED)
model_dectree2.fit(X_train, y_train)
y_pred_dectree2 = model_dectree2.predict(X_valid)
print(f"Точность модели DecisionTreeRegressor по метрике MAPE: {(mape(y_valid, y_pred_dectree2))*100:0.2f}%")

Как мы видим, модель Decision Tree с гиперпараметрами работает хуже, чем без гиперпараметров

In [ ]:
# Обучим модель Random Forest
model_randfor = RandomForestRegressor(n_estimators = 100, random_state = RANDOM_SEED)
model_randfor.fit(X_train, y_train)  
y_pred_randfor = model_randfor.predict(X_valid)
print(f"Точность модели Random Forest по метрике MAPE: {(mape(y_valid, y_pred_randfor))*100:0.2f}%")

In [ ]:
# Поробуем настроить гиперпараметры для модели Random Forest
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3, 5, 10],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search_randf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search_randf .fit(X_train, y_train)

In [ ]:
grid_search_randf.best_params_

In [ ]:
# Обучим модель Random Forest с гиперпараметрами
model_randfor2 = RandomForestRegressor(bootstrap = False, max_depth = 110, max_features = 10, min_samples_leaf = 3, min_samples_split = 8, n_estimators = 100, random_state = RANDOM_SEED)
model_randfor2.fit(X_train, y_train)  
y_pred_randfor2 = model_randfor2.predict(X_valid)
print(f"Точность модели Random Forest по метрике MAPE: {(mape(y_valid, y_pred_randfor2))*100:0.2f}%")

Как видим, модель RandomForestRegressor с гипермараметрами работает намного хуже, чем модель без гиперпараметров.

Применим модель RandomForestRegressor к тестовым данным и загрузим Submission в соревнование

In [ ]:
predict_submission_randfor = model_randfor.predict(X_2)

In [ ]:
sample_submission['price'] = predict_submission_randfor
sample_submission.to_csv(f'submission_randfor.csv', index=False)
sample_submission.head(10)

На странице соревнования получили **MAPE 22.3%**. К сожалению результат алгоритма CatBoosting улучшить не удалось

#### Теперь попробуем обучить какой-нибудь ансамбль

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
# Определим базовые модели
level0 = list()
level0.append(('knn', KNeighborsRegressor()))
level0.append(('rand', RandomForestRegressor()))
level0.append(('cart', DecisionTreeRegressor()))
level0.append(('svm', SVR()))

In [ ]:
# Определим мета-модель
level1 = LinearRegression()

In [ ]:
# Определим модель для ансамбля и обучим ее
model_ens = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model_ens.fit(X_train, y_train)  
y_pred_ens = model_ens.predict(X_valid)
print(f"Точность модели Ансамбля по метрике MAPE: {(mape(y_valid, y_pred_ens))*100:0.2f}%")

#### Попробуем улучшить результат с помощью логарифмирования таргета

In [ ]:
# Определим модель для ансамбля и обучим ееVersion 9
model_ens = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)
model_ens.fit(X_train, np.log(y_train))  
y_pred_ens = np.exp(model_ens.predict(X_valid))
print(f"Точность модели Ансамбля по метрике MAPE: {(mape(y_valid, y_pred_ens))*100:0.2f}%")

Применим модель Ансамбля к тестовым данным и загрузим Submission в соревнование

In [ ]:
predict_submission_ens = np.exp(model_ens.predict(X_2))

In [ ]:
sample_submission['price'] = predict_submission_ens
sample_submission.to_csv(f'submission_ens2.csv', index=False)
sample_submission.head(10)

На странице соревнования получаем **MAPE 20 - 22%**. К сожалению результат алгоритма CatBoosting улучшить не удалось

## Вывод:
Мы перепробовали множество разных алгоритмов, но, к сожалению, результат алгоритма CatBoosting в 20% на странице соревнования на Kaggle улучшить не удалось. Возможно, результат можно было бы улучшить за счет обогащения датасета как новыми данными так и новыми признаками. Но поскольку парсинг данных занимает достаточно много времени, я бы пока остановился на этом. MAPE в 20% на мой взгляд - достаточно приемлимый результат